In [2]:
from astroquery.gaia import Gaia
import pandas as pd
import numpy as np
from astropy.table import vstack, Table

import pickle
import os
computer = os.getenv('COMPUTERNAME')

Created TAP+ (v1.2.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443
Created TAP+ (v1.2.1) - Connection:
	Host: geadata.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443


## Read in our catalog

Read in the csv file listing all observations, downloaded from the google drive (created by Alana Sanchez). Most important column is 'starname', which is the 2MASS ID name with, for binaries, some extra letters appended to indicate which component of the binary. We will break apart into just the number part of the 2MASS ID, and to the component. Later, we will match each component of the binary to a separate entry in the Gaia catalog.

In [22]:
iddf = pd.read_csv('alana_list_09Mar2019.csv',  delimiter=',', encoding="ISO-8859-1")
df = iddf.rename(index=str, columns={'Star Name (2MASS)':'starname'}) # rename the column so its easier

# this creates empty columns for entries we are about to add
df['twomass'] = ["" for x in range(len(df['Night']))]
df['twomass_comp'] = ["" for x in range(len(df['Night']))]

for index, r in df.iterrows():
    value = r['starname'][1:17] # don't want the "J", and the 2MASS ID goes until character 17
    # set this to column 'twomass'
    value = r['starname'][17:] # anything after character 17 is a component for a binary
    # set this to the column 'twomass_comp'

df

,starname,Night,Current Status,Issues,ERN Comments,Anyone working on this?,Unnamed: 6,twomass,twomass_comp
0,J05581716-0438012,20110322,Through code,NaN,NaN,NaN,NaN,,
1,J06171064+0507024,20110322,Through code,NaN,NaN,NaN,NaN,,
2,J06521804-0511241,20110322,Through code,NaN,NaN,NaN,NaN,,
3,J08103429-1348514,20110322,NaN,"may be issues with 94-95, profile is doubled (...",NaN,NaN,NaN,,
4,J08124088-2133056,20110322,Through code,NaN,NaN,NaN,NaN,,
...,...,...,...,...,...,...,...,...,...
260,J04231246+1542462,20121230,Through code,NaN,HIP16322,NaN,NaN,,
261,J04243151+1355430,20121230,Through code,NaN,HIP16322,NaN,NaN,,
262,J04300417+1604079,20121230,Through code,NaN,HIP16322,NaN,NaN,,
263,J04382771+1600109,20121230,Through code,NaN,HD43607,NaN,NaN,,


# Stars that turned out to be binaries

Sometimes when we observe a star, it turns out to be a binary! This happens not infrequently when observing low-mass stars, because usually they've previously only been observed a low-spatial resolution, all-sky survey. The origin catalog for this work was the 2MASS survey. The CCD in the 2MASS survey had pixels that were 2" across, so if two stars fell on the same pixel we probably wouldn't know there were two of them. When we observed at the telescope, we had higher spatial resolution, and could see that there were two stars. We usually then observed both of them. I manually went through all of my notes (which had little diagrams saying which star was to which side of the other) and identified which star in Gaia each of them matched. Thw two stars in a binary are called "components". You'll see, for example, that the 'SE' component has an RA and Dec that places it to the SE of the other component; I also looked at brightness when one component was brighter than the other.

This file also contains the stars that didn't get matched automatically when I tried to do my Gaia cross-match, but your cross-match will hopefully be more robust!

In [13]:
# this is the CSV file that contains the splits. 'starname' matches the
# name in the big catalog of sources. 'gaiasource' is the Gaia DR2 ID
# (at least, I'm pretty sure that's the data release I used!).
# 'gaiasource' is a string, but will probably need to be int64

dupdf = pd.read_csv('twomass_splits.csv',  delimiter=',')
dupdf

,starname,Unnamed: 1,gaiasource,splitra,splitdec,Unnamed: 5,direction
0,J01230055-1257298SE,south and east and bright,'2456361654226797440',20.754572,-12.958509,NaN,south and east and bright
1,J01230055-1257298NW,NaN,'2456361654225919488',20.754368,-12.958188,NaN,NaN
2,J07575485-6017584E,south and east and bright,'5291028284195365632',119.483203,-60.299032,9.202524,south and east and bright
3,J07575485-6017584W,NaN,'5291028284195365248',119.481938,-60.298763,11.616670,NaN
4,J17195815-0553043E,east and a little north and faint,'4361366292101060096',259.992742,-5.885322,NaN,NaN
5,J17195815-0553043W,west and bright,'4361366292103812480',259.992440,-5.885472,NaN,NaN
6,J07212226-2820348E,east a little south and very faint,'5606673093009210624',110.343798,-28.343252,NaN,5600K
7,J07212226-2820348W,west a little north and bright,'5606673093014025728',110.342804,-28.343031,NaN,note plxs do not match
8,J08103429-1348514,no 2mass match,'5725122965270676864',NaN,NaN,NaN,NaN
9,J14294291-6240465,no 2mass match,'5853498713160606720',NaN,NaN,NaN,NaN


In [14]:
# merge duplicates catalog with main df

df = df.merge(dupdf, how='left', right_on='starname', left_on='starname')


## Query Gaia - 2MASS cross-match for all sources

Primary cross-match is on official 2MASS cross-match. Then I do the splits separately. I merge each of the resulting catalogs with our original target list separately, requiring a Gaia match. In this way the two catalogs are non-overlapping (one contains single stars with matches, the other contains the double stars + those without matches). 

In [15]:
debug=False
if debug: ## make a list of only 10 to heck
    idlist ="'" + "', '" .join(df['twomass'][231:242]) + "'"    # first and last are split
else: ## do the whole thing
    idlist ="'" + "', '" .join(df['twomass']) + "'"

# this queries Gaia data for the Gaia DR2 data, searching based on the 2MASS ID
# need to update to Gaia DR3!
job = Gaia.launch_job_async("SELECT * \
    FROM gaiadr2.gaia_source AS g, gaiadr2.tmass_best_neighbour AS tbest \
    WHERE g.source_id = tbest.source_id \
    AND tbest.original_ext_source_id IN (" + idlist +")", dump_to_file=False)
print(cmd)

# get the results
gaiaData1 = job.get_results().to_pandas()


# may need to manipulate the 'original_ext_source_id' column (which contains the Gaia ID)


# this star had a double match when I tried this (it might not anymore!)
# this is how I dropped the bad one
#bad = gaiaData1.index[(gaiaData1['original_ext_source_id']=='07171706-0501031') & (gaiaData1['source_id'] == 3059246454088275840)]
#gaiaData1 = gaiaData1.drop(bad)


# check for missing matches
test1 = df.merge(gaiaData1, how='left', left_on='twomass', right_on='original_ext_source_id')

# merged with Alana's table (single stars with Gaia match only)
gaiaData1 = df.merge(gaiaData1, how='right', left_on='twomass', right_on='original_ext_source_id')

INFO: Query finished. [astroquery.utils.tap.core]


In [4]:
# select out those stars that had manual entries in the twomass_splits table
idlist = ", ".join([str(x) for x in df['gaiasource'][df['gaiasource'].notnull()]])

# also needs to be updated to Gaia DR3
# the command is similar to the above but queries on Gaia DR2 ID instead of 2MASS ID
cmd = "SELECT * \
    FROM gaiadr2.gaia_source AS g \
    WHERE g.source_id IN (" + idlist +")"
job = Gaia.launch_job_async(cmd, dump_to_file=False)

# get the data
gaiaData2 = job.get_results().to_pandas()


# merged with master table (split stars with Gaia match only)
gaiaData2 = df.merge(gaiaData2, how='right', right_on='source_id', left_on='gaiasource')

SELECT *     FROM gaiadr2.gaia_source AS g     WHERE g.source_id IN (5725122965270676864, 5853498713160606720, 3195919322830293760, 3117120863523946368, 5260451999698559872, 6109949904687424640, 2368293487260807040, 3340965419296002944, 2512629230496996992, 5164137461165628032, 2358524597030794112, 4947513192089475584, 5153091836072107008, 4838609039260040704, 4941699593078093952, 4288649399672832, 4361366292101060096, 4361366292103812480, 4177855052651068928, 1814240507054682880, 6887029163498137856, 6894054664842632448, 5901094750438455296, 6768500534421789440, 5605469398353836544, 5471345889049823744, 2456361654225919488, 2456361654226797440, 5291028284195365632, 5291028284195365248, 5302788969813884160, 3864686545890130432)
Query finished.


### Check for failures

See if a star didn't make it through. Don't remember exactly what this does!

In [6]:
failed = test1[test1['source_id'].notnull()==False]['starname'].values
passed = test2[test2['source_id'].notnull()]['starname']
for f in failed:
    if np.any(f==passed):
        pass
    else:
        print(f)

J10442131-6112385


### Final Gaia catalog

Which in theory contains every star from our original target list and only once for each! HOWEVER, there are still duplicates because some objects were observed on two nights. Still need to sort this out.

In [7]:
gaiaData = pd.concat([gaiaData1, gaiaData2])
gaiaData[['starname','twomass','Night','source_id','original_ext_source_id', 'phot_g_mean_mag']][205:]

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if __name__ == '__main__':


,starname,twomass,Night,source_id,original_ext_source_id,phot_g_mean_mag
205,J04235666+1838201,04235666+1838201,20121228,47881811540636800,04235666+1838201,15.481036
206,J04251652+1618086,04251652+1618086,20121228,3313579505227558528,04251652+1618086,15.028309
207,J07171706-0501031,07171706-0501031,20121228,3059246454092369024,07171706-0501031,11.874182
208,J07324858-4912430,07324858-4912430,20121228,5506284726451372288,07324858-4912430,11.028179
209,J07401183-4257406,07401183-4257406,20121228,5535361753834861312,07401183-4257406,12.044649
210,J08193570-4237008,08193570-4237008,20121228,5527125277861239040,08193570-4237008,11.549067
211,J08415977-0134083,08415977-0134083,20121228,3072544153868881792,08415977-0134083,12.749329
212,J08533619-0329321,08533619-0329321,20121228,5761985432616501376,08533619-0329321,15.908699
213,J10582800-1046304,10582800-1046304,20121228,3758629475341196672,10582800-1046304,12.767668
214,J04153367+1542226,04153367+1542226,20121229,3312197934506930944,04153367+1542226,10.431786


In [ ]:
# save the data

# Stellar Parameters

In [2]:
catalogData = pd.read_pickle('FIRE_catalogData.pkl')

### Masses from Mk

In [18]:
def Dist(plx, e_plx):
    
    d = 1000./plx
    ed = 1000./plx**2*e_plx
    return d, ed

def AbsMag(mag, dist, e_mag, e_dist):
    
    absm = mag - 5*np.log10(dist) + 5
    eabsm = np.sqrt( (e_mag)**2 + (5./np.log(10)*e_dist/dist)**2 )
    return absm, eabsm

In [3]:
from mk_mass import posterior

k,ek  = 5.3,0.02
dist  = 14.55
edist = 0.13
feh,efeh=0.5,0.1
mass      = posterior(k,dist,ek,edist)
mass_feh  = posterior(k,dist,ek,edist,feh,efeh)
print ("Mass=%6.4f+/-%6.4f" % (np.median(mass),np.std(mass)))
print ("Mass=%6.4f+/-%6.4f" % (np.median(mass_feh),np.std(mass_feh)))

# the feh makes no difference, and has larger errors...

ModuleNotFoundError: No module named 'mk_mass'

### Luminosities from Mann et al. V and J 

In [22]:
def Mann_vjLum(mv, mj, dist, e_mv, e_mj, e_dist):
    absmj, e_absmj = AbsMag(mj, dist, e_mj, e_dist)
    #absmv, eabsmv = AbsMag(mv, dist) ## for absV-based version

    co = [0.8694, 0.3667, -0.02920]
    c = list(co)
    #c = [0.5817, -0.4168, -0.08165, 4.084e-3] ## for absV-based version
    
    c.reverse()
    p = np.poly1d(c)
    
    absbol = absmj + p(mv-mj)
    mbol = 4.74 # https://arxiv.org/abs/1510.06262
    lbol = np.power(10., -0.4*(absbol-mbol))
    
    # calculate error
    e_absbol = np.sqrt( (e_absmj)**2. \
                       + ((co[1] + 2.*co[2]*(mv-mj))*e_mv)**2. \
                       + ((co[1] + 2.*co[2]*(mv-mj))*e_mj)**2 \
                       + 0.016**2.)
    e_lbol = np.log(10.)*lbol*0.4*e_absbol
    
    return lbol, e_lbol


d, e_d = Dist(67.1548125858965, 0.05059913591456251)
print Mann_vjLum(15.26, 10.046, d, 0.03,  0.023, e_d) ## not close


def Mann_vjFeHLum(mv, mj, dist, feh, e_mv, e_mj, e_dist, e_feh):
    absmj, e_absmj = AbsMag(mj, dist, e_mj, e_dist)
    co = [0.8879, 0.3563, -0.02791, -0.04857]
    c = list(co)
    c.reverse()
    p = np.poly1d(c[1:])
    
    # calculate lbol
    absmbol = absmj + p(mv-mj) + co[3]*feh
    mbol = 4.74
    lbol = np.power(10., -0.4*(absmbol-mbol))

    # calculate error
    e_absmbol = np.sqrt( (e_absmj)**2 \
                + (co[1] + 2.*co[2]*(mv-mj))**2*e_mj**2 \
                + (co[1] + 2.*co[2]*(mv-mj))**2*e_mv**2 \
                + (co[3]*e_feh)**2 \
                + 0.012**2.)
    e_lbol = np.log(10.)*0.4*lbol*e_absmbol
        
    return lbol, e_lbol


(0.0026817994318676939, 6.9565667447177526e-05)
(0.090610366521928609, 0.0025137920870558811)
(0.090606263968969758, 0.0023519024942970585)


### Save inferred parameters

In [ ]:
# TO DO!
# actually want to use his code with the errors
catalogData['mann_mkmass'] = np.zeros_like(catalogData['ks_m'])*np.nan
catalogData['benedict_mkmass'] = np.zeros_like(catalogData['ks_m'])*np.nan
for index, r in catalogData.iterrows():
    if np.isfinite(r['ks_m']) & (r['parallax_over_error'] > 3.):
        value = mann_mass(r['ks_m'], 1000./r['parallax'])
        catalogData.set_value(index,'mann_mkmass',value)
        value = benedict_mass(r['ks_m'], 1000./r['parallax'])
        catalogData.set_value(index,'benedict_mkmass',value)
